In [1]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tensorflow as tf

In [310]:
# data_excel에서 raw_data set 을 불러옵니다. 
raw_data = pd.read_excel('Data2.xlsx', sheet_name='7차SIZEKOREA_ROWDATA')
raw_data

,ⓞ_02_성별,ⓞ_06_나이_반올림,ⓞ_12_골격근량,ⓞ_13_체지방량,ⓞ_14_체수분,ⓞ_15_단백질,ⓞ_16_무기질,ⓞ_17_BMI,ⓞ_18_체지방율,ⓞ_19_복부지방율,...,①_124_무릎아래둘레,①_125_장딴지둘레,①_126_종아리최소둘레,①_127_발목최대둘레,①_128_발목너비,①_129_발뒤꿈치너비,①_130_위팔둘레,①_131_팔꿈치둘레,①_132_손목둘레,①_133_머리위로뻗은주먹높이
0,남,25,33.2,13.6,42.9,11.8,3.8,23.924112,18.9,0.9,...,325.0,370.0,209.0,241.0,50.0,62.0,325.0,286.0,153.0,2013.0
1,남,28,44.5,28.7,56.5,15.5,5.5,31.608187,27.0,0.9,...,396.0,439.0,260.0,295.0,62.0,77.0,395.0,318.0,179.0,2160.0
2,남,19,36.5,5.6,46.6,12.7,4.2,22.718784,8.1,0.8,...,350.0,380.0,232.0,268.0,54.0,63.0,313.0,291.0,166.0,2062.0
3,남,20,35.2,7.0,45.1,12.3,4.0,22.309190,10.3,0.8,...,330.0,368.0,219.0,257.0,58.0,68.0,303.0,295.0,163.0,2020.0
4,남,22,40.4,11.6,51.5,14.0,4.8,23.904027,14.1,0.9,...,362.0,391.0,230.0,279.0,62.0,68.0,304.0,291.0,167.0,2148.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,여,20,22.2,18.4,30.2,8.0,3.0,20.478031,30.9,0.8,...,355.0,368.0,207.0,234.0,48.0,59.0,265.0,244.0,149.0,1992.0
6416,여,20,21.5,15.2,29.1,7.8,2.8,20.816145,27.7,0.8,...,330.0,346.0,190.0,229.0,53.0,63.0,265.0,271.0,144.0,1878.0
6417,여,44,18.6,15.4,25.5,6.8,2.3,22.031042,30.7,0.9,...,321.0,334.0,193.0,231.0,53.0,60.0,260.0,237.0,140.0,1753.0
6418,여,20,17.6,11.3,24.4,6.5,2.3,18.002193,25.4,0.8,...,341.0,320.0,200.0,231.0,54.0,59.0,222.0,222.0,139.0,1767.0


In [311]:
# BMI지수, 체지방량, 체지방율을 이용하여 몸무게와 키를 구해주기 위해  BMI지수, 체지방량, 체지방률을 따로 빼줍니다.
df1 = raw_data.loc[:,['ⓞ_13_체지방량','ⓞ_17_BMI','ⓞ_18_체지방율']]

# 결측치를 제거해 줍니다.
df1 = df1.dropna( how = 'any')
# 계산의 편의를 위해서 열의 이름을 바꿔줍니다.
df1.columns = ['a', 'BMI', 'c']
# 몸무게 = 체지방량/체지방율 * 100
df1['Weight'] = df1['a']/df1['c']*100
# 키 = (몸무게/BMi)^(1/2)
df1['Height'] = np.sqrt(df1['Weight']/df1['BMI'])
# a, c 열을 삭제해준다.
df1 = df1.drop(['a', 'c'], axis=1)

Raw_data_length = len(df1)
df1

,BMI,Weight,Height
0,23.924112,71.957672,1.734286
1,31.608187,106.296296,1.833831
2,22.718784,69.135802,1.744452
3,22.309190,67.961165,1.745374
4,23.904027,82.269504,1.855171
...,...,...,...
6415,20.478031,59.546926,1.705240
6416,20.816145,54.873646,1.623610
6417,22.031042,50.162866,1.508946
6418,18.002193,44.488189,1.572026


In [ ]:
# BMI encode 를 작성해줍니다. 기준은 BMI table 을 기준으로 작성해 주었습니다..  18.5, 23, 25 
df1['BMI_encode'] =  np.where(df1['BMI'] <= 18.5, 0,
                                 np.where(df1['BMI'] <= 23, 1,
                                         np.where(df1['BMI'] <= 25, 2, 3)))

# Condition 도 행마다 상태별로 0, 1, 2, 3 으로 만들어준다.
df_condition = pd.concat([df1, df1, df1, df1])
df_condition['Condition'] = 0
df_condition.iloc[:Raw_data_length]['Condition'] = 0
df_condition.iloc[Raw_data_length:Raw_data_length*2]['Condition'] = 1
df_condition.iloc[Raw_data_length*2:Raw_data_length*3]['Condition'] = 2
df_condition.iloc[Raw_data_length*3:Raw_data_length*4]['Condition'] = 3
df_condition['Condition'].value_counts()

In [247]:
# BMI를 기준으로 정렬해줍니다.
df_sort = df_condition.sort_values(by='BMI')
# index를 초기화 해줍니다.
data = df_sort.reset_index().drop('index', axis = 1)
data

,BMI,Weight,Height,BMI_encode,Condition
0,14.703798,64.963504,2.101938,0,3
1,14.703798,64.963504,2.101938,0,0
2,14.703798,64.963504,2.101938,0,2
3,14.703798,64.963504,2.101938,0,1
4,14.808706,41.293532,1.669869,0,1
...,...,...,...,...,...
25599,41.303140,95.283019,1.518855,3,1
25600,43.277877,110.153257,1.595386,3,1
25601,43.277877,110.153257,1.595386,3,2
25602,43.277877,110.153257,1.595386,3,3


In [268]:
# 조건절을 걸어 Mountain_encode를 생성해준다.
# BMI와 Condition을 기준으로 Mountain을 추천하는 코드를 만들어준다. 
# BMI_encode 가 0 일때 기분이 0이거나 1이면 Mountain 1(200m)이하의 산을 추천 해준다.
Condition0_1 = (data['BMI_encode']==0) & ( (data['Condition']==0)|(data['Condition']==1) )
Condition0_2 = (data['BMI_encode']==0) & ( (data['Condition']==2)|(data['Condition']==3) )
Condition1_1 = (data['BMI_encode']==1) & ( (data['Condition']==0)|(data['Condition']==1) )
Condition1_2 = (data['BMI_encode']==1) & ( (data['Condition']==2)|(data['Condition']==3) )
Condition2_1 = (data['BMI_encode']==2) & ( (data['Condition']==0)|(data['Condition']==1) )
Condition2_2 = (data['BMI_encode']==2) & ( (data['Condition']==2)|(data['Condition']==3) )
Condition3_1 = (data['BMI_encode']==3) & ( (data['Condition']==0)|(data['Condition']==1) )
Condition3_2 = (data['BMI_encode']==3) & ( (data['Condition']==2)|(data['Condition']==3) )
data['Mountain_encode'] = np.where(Condition0_1, 0,
                                   np.where(Condition0_2, 2,
                                            np.where(Condition1_1, 1,
                                                     np.where(Condition1_2, 3,
                                                              np.where(Condition2_1, 1,
                                                                       np.where(Condition2_2, 2,
                                                                                np.where(Condition3_1, 0, 2))))))) 

In [269]:
data['Mountain_encode'].value_counts()

1    8368
2    7082
3    5720
0    4434
Name: Mountain_encode, dtype: int64

In [277]:
# Xdata에 Weight, Height, Condition 을 지정해주고, ydata에 Mountain_encode 를 지정해준다. 
xdata = data.loc[:, ['Weight', 'Height', 'Condition']]
ydata = data.loc[:,['Mountain_encode']]


In [278]:
e = LabelEncoder()
e.fit(ydata)
y = e.transform(ydata)
y_encoded = tf.keras.utils.to_categorical(y)

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [279]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(xdata, y_encoded, test_size=0.2, random_state=1)

In [280]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=1)
model.fit(x_train, y_train)

DecisionTreeClassifier(random_state=1)

In [281]:
pred = model.predict(x_test)
pred

array([[0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]], dtype=float32)

In [282]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, pred)
print('accuracy:', acc)

accuracy: 0.9960945127904706


In [283]:
pred_soyeon = model.predict([[48,1.61,3]])
pred_soyeon

array([[0., 0., 1., 0.]], dtype=float32)

In [284]:
pred_jiyeon = model.predict([[52,1.57,0]])
pred_jiyeon

array([[0., 1., 0., 0.]], dtype=float32)

In [285]:
pred_yeongki = model.predict([[73,1.83,2]])
pred_yeongki

array([[0., 0., 0., 1.]], dtype=float32)

In [286]:
pred_junho = model.predict([[83,1.78,0.5]])
pred_junho.argmax()

0

In [287]:
pred_minjae = model.predict([[73,1.73,1]])
pred_minjae.argmax()

1

In [288]:
import pickle
import joblib
# 객체를 pickled binary file 형태로 저장한다
file_name = 'model_01.pkl'
joblib.dump(model, file_name)

# pickled binary file 형태로 저장된 객체를 로딩한다
file_name = 'model_01.pkl'
model = joblib.load(file_name)

def predict(item):
    file_name = 'model_01.pkl'
    model = joblib.load(file_name)
    pred = model.predict([item])
    return pred.argmax()

def input_info() :
    print('----------정보 입력 ------------')
    Weight = input('몸무게를 입력해주세요 :' )
    Height = input('키를 입력해주세요 : ')
    Condition = input('현재 상태를 입력해주세요 : ')
    item = [float(Weight), float(Height), float(Condition)]
    return df_Mt[df_Mt['mountain_encode'] == predict(item)].iloc[:,0].values

In [309]:
input_info()

----------정보 입력 ------------
몸무게를 입력해주세요 :10
키를 입력해주세요 : 10
현재 상태를 입력해주세요 : 1


array(['굴봉산', '호암산', '일자산', '대모산', '성산', '노고산'], dtype=object)

In [ ]:
#         try :
#             while True :
#                 a1 = pickle.load(fp) #객체가 몇개 들어갈지 모르니까 loads로 읽어온다.
#                 animal_list.append(a1)
#         except :
#             pass

In [317]:
# 산데이터 불러와서 뽑기
df_Mt = pd.read_excel('Data2.xlsx', sheet_name='산')
df_Mt

,Unnamed: 0,높이,구,0~3
0,굴봉산,66,마포구,11
1,호암산,78,양천구,17
2,일자산,79,마포구,11
3,대모산,81,성동구,14
4,성산,95,강남구,0
5,노고산,99,강서구,3
6,금호산,106,동대문구,10
7,백련산,106,마포구,11
8,오패산,108,구로구,6
9,낙산,114,노원구,8


In [316]:
# 산데이터 encoding 하기
df_Mt['mountain_encode'] = np.where(df_Mt['높이'] >= 500, 3,
                                 np.where(df_Mt['높이'] >= 200, 2,
                                         np.where(df_Mt['높이'] >= 100, 1, 0)))
# 조건에 맞는 산이름 가져오기
df_Mt[df_Mt['mountain_encode'] == 3].iloc[:,0].reset_index().drop('index', axis=1).iloc[:,0]

0    개화산
1    용마산
2    대모산
3    초안산
4    벽오산
5    봉화산
6    금호산
7    남한산
8    수락산
Name: Unnamed: 0, dtype: object

In [301]:
# mountain_encode 가 0 인 조건 찾기
df_Mt['mountain_encode'] == 0 

0      True
1      True
2      True
3      True
4      True
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
Name: mountain_encode, dtype: bool

In [302]:
# df1에서 조건에 맞는 데이터를 필터링
df_Mt[df_Mt['mountain_encode'] == 0]

,Unnamed: 0,높이,구,0~3,mountain_encode
0,굴봉산,66,마포구,11,0
1,호암산,78,양천구,17,0
2,일자산,79,마포구,11,0
3,대모산,81,성동구,14,0
4,성산,95,강남구,0,0
5,노고산,99,강서구,3,0


In [303]:
# 조건에 맞는 산이름 가져오기
df_Mt[df_Mt['mountain_encode'] == 3].iloc[:,0].reset_index().drop('index', axis=1).iloc[:,0]

0    개화산
1    용마산
2    대모산
3    초안산
4    벽오산
5    봉화산
6    금호산
7    남한산
8    수락산
Name: Unnamed: 0, dtype: object